In [33]:
from rdkit import Chem
from rdkit.Chem import AllChem
from ase.io import read
from quippy.descriptors import Descriptor
import pickle



In [27]:
def molfile_to_xyz(molfile:str):
    temp_file = 'temp2.mol'
    with open(temp_file, 'w+') as f:
        f.writelines(molfile) #write molfile to temporary file
    mol = Chem.MolFromMolFile(temp_file) # Read temp file into RDKit molecule
    print(type(mol))
    #mol = Chem.AddHs(mol)# post-process molecule
    AllChem.EmbedMolecule(mol)
    Chem.MolToMolFile(mol, temp_file)    # Write RDKit molecule to a temporary file
    molecule = read('temp.mol') # Read the temporary file into ASE Atoms object
    return molecule
FILE = 'temp.mol'
with open(FILE, 'r+') as f:
    content = f.readlines()
print(content)
mol = molfile_to_xyz(content)

['\n', '     RDKit          3D\n', '\n', '  8  7  0  0  0  0  0  0  0  0999 V2000\n', '   -0.7533   -0.0093    0.0601 C   0  0  0  0  0  0  0  0  0  0  0  0\n', '    0.7504   -0.0263    0.0011 C   0  0  0  0  0  0  0  0  0  0  0  0\n', '   -1.1589    0.1177    1.0753 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '   -1.0772    0.8427   -0.5977 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '   -1.1758   -0.9182   -0.4328 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '    1.0733   -0.5749   -0.9198 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '    1.2143   -0.4556    0.9130 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '    1.1271    1.0239   -0.0993 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '  1  2  1  0\n', '  1  3  1  0\n', '  1  4  1  0\n', '  1  5  1  0\n', '  2  6  1  0\n', '  2  7  1  0\n', '  2  8  1  0\n', 'M  END\n']
<class 'rdkit.Chem.rdchem.Mol'>


[14:18:00] Molecule does not have explicit Hs. Consider calling AddHs()


In [34]:
Z = 6 
cutoff = 5; dc = 0.5; sigma = 0.5
zeta = 6
soap = {"C": 'soap_turbo alpha_max={8 8 8} l_max=8 rcut_soft=%.4f rcut_hard=%.4f atom_sigma_r={%.4f %.4f %.4f} atom_sigma_t={%.4f %.4f %.4f} \
               atom_sigma_r_scaling={0. 0. 0.} atom_sigma_t_scaling={0. 0. 0.} radial_enhancement=1 amplitude_scaling={1. 1. 1.} \
               basis="poly3gauss" scaling_mode="polynomial" species_Z={1 6 8} n_species=3 central_index=2 central_weight={1. 1. 1.} \
               compress_mode=trivial' % (cutoff-dc, cutoff, *(6*[sigma])),
        "O": 'soap_turbo alpha_max={8 8 8} l_max=8 rcut_soft=%.4f rcut_hard=%.4f atom_sigma_r={%.4f %.4f %.4f} atom_sigma_t={%.4f %.4f %.4f} \
               atom_sigma_r_scaling={0. 0. 0.} atom_sigma_t_scaling={0. 0. 0.} radial_enhancement=1 amplitude_scaling={1. 1. 1.} \
               basis="poly3gauss" scaling_mode="polynomial" species_Z={1 6 8} n_species=3 central_index=3 central_weight={1. 1. 1.} \
               compress_mode=trivial' % (cutoff-dc, cutoff, *(6*[sigma]))}

desc=Descriptor(soap["C"])

# First we create the descriptor object

Z_O = 8 
desc_O = Descriptor(soap["O"])

def xyz_to_soap(mol):
    elements = []
    if ("C" in mol.symbols) == True:
        descMol = desc.calc(mol) #descriptor for each molecule

        if 'data' in descMol:
           desc_data = descMol['data'] #get the data from the descriptor object if exist
           print(len(desc_data))
           for element in desc_data:
               elements.append(element)
    return elements

els = xyz_to_soap(mol)


2


In [50]:
import os
MODEL_O = '../MLmodels/XPS_GPR_O1s.pkl'
MODEL_C = '../MLmodels/XPS_GPR_C1s_new.pkl'
print(os.path.exists(MODEL_C))
def soap_to_BE(soap):
    model = pickle.load(open(MODEL_O, 'rb'))

    be = model.predict(soap, return_std = True)
    print(be)
    return be

soap_to_BE(els)

False
(array([541.63035912, 541.60622392]), array([1.80196885, 1.80147875]))


(array([541.63035912, 541.60622392]), array([1.80196885, 1.80147875]))